# This is how I read the WOA 1° monthly climatologies

you'll need a python env, if you don't have one, try Anaconda, and install relevant packages like:

> conda install -c anaconda numpy

in a terminal

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob

In [2]:
# this collects all monthly files
NitrateNCDFfiles = glob.glob('../WOA2018/Nitrate/*.nc', recursive=True)

# and opens them 
WOAnitrate = xr.open_mfdataset(NitrateNCDFfiles,combine='by_coords',decode_times=False)

# For other variables:

In [3]:
#TempNCDFfiles = glob.glob('../WOA2018/Temperature/*.nc', recursive=True)
#WOAtemp = xr.open_mfdataset(TempNCDFfiles,combine='by_coords',decode_times=False)

In [4]:
#PhosphateNCDFfiles = glob.glob('WOA2018/Phosphate/*.nc', recursive=True)
#WOAphosphate = xr.open_mfdataset(PhosphateNCDFfiles,combine='by_coords',decode_times=False)

In [5]:
#SilicateNCDFfiles = glob.glob('WOA2018/Silicate/*.nc', recursive=True)
#WOAsilicate = xr.open_mfdataset(SilicateNCDFfiles,combine='by_coords',decode_times=False)

In [6]:
#WOAsilicate.variables

## The function below extracts data profiles in location

for now set up for Temperature, Nutrients or Phosphate & Silicate, you'll have to add other vars below

In [7]:
def WOADatInLocation(WOAfile, lats=40,lons=-20, WOAvar='Temp', justValPerDepth=True):
    """This function returns the xarray of Temperature in a specific location"""
    
    WOAdat = WOAfile.sel(nbounds=0)
    WOA_df = WOAdat.sel(lat=lats,lon=lons, method='nearest').to_dataframe()
    
    if WOAvar=='Temp':
        var='t_an'
        WOA_df.index = pd.MultiIndex.from_arrays([WOA_df.index.get_level_values(level='depth'), [i for j in range(0,57) for i in range(1,13)]])
        WOA_df.index.names = ['depth','time']
    elif WOAvar=='N' or WOAvar=='N_above':
        var='n_an'
        WOA_df.index = pd.MultiIndex.from_arrays([WOA_df.index.get_level_values(level='depth'), [i for j in range(0,43) for i in range(1,13)]])
        WOA_df.index.names = ['depth','time']
    elif WOAvar=='P' or WOAvar=='P_above':
        var='p_an'
        WOA_df.index = pd.MultiIndex.from_arrays([WOA_df.index.get_level_values(level='depth'), [i for j in range(0,43) for i in range(1,13)]])
        WOA_df.index.names = ['depth','time']
    elif WOAvar=='Si' or WOAvar=='Si_above':
        var='i_an'
        WOA_df.index = pd.MultiIndex.from_arrays([WOA_df.index.get_level_values(level='depth'), [i for j in range(0,43) for i in range(1,13)]])
        WOA_df.index.names = ['depth','time']
    
    if justValPerDepth==True:
        return WOA_df[[var]]#.reset_index()
    else:
        return WOA_df#.reset_index()

In [8]:
WOADatInLocation(WOAnitrate,WOAvar='N')

n_an
depth time           
0.0   1      2.256596
      2      3.053565
      3      1.944138
      4      1.735271
      5      1.208379
...               ...
800.0 8     17.098936
      9     16.831354
      10    17.391563
      11    17.785587
      12    18.272003

[516 rows x 1 columns]

In [9]:
# get data in location
WOA = WOADatInLocation(WOAnitrate,lats=40,lons=-20,WOAvar='N')

In [10]:
def interpolateWOAprofiles(WOAdat,var):
    """this function interpolates the WOA profiles to a resolution of 1 meter"""
    WOAgrid = WOAdat.unstack(level=1)
    if var=='Temp':
        WOAgrid1 = WOAgrid.reindex(range(0,1501))
        return WOAgrid1.interpolate(method='linear')
    elif var=='N' or var=='P' or var=='Si':
        WOAgrid1 = WOAgrid.reindex(range(0,801))
        return WOAgrid1.interpolate(method='linear')
    elif var=='N_above' or var=='P_above' or var=='Si_above':
        WOAgrid1 = WOAgrid.reindex(range(0,801))
        return WOAgrid1.interpolate(method='linear')

In [11]:
#interpolate profile
WOA_int = interpolateWOAprofiles(WOA,'N')

In [12]:
WOA_int

n_an                                                         \
time          1          2          3          4          5          6    
depth                                                                     
0       2.256596   3.053565   1.944138   1.735271   1.208379   0.043236   
1       2.396459   3.038053   1.937282   1.735264   1.097125   0.043524   
2       2.536322   3.022541   1.930426   1.735258   0.985871   0.043812   
3       2.676186   3.007029   1.923571   1.735251   0.874618   0.044100   
4       2.816049   2.991517   1.916715   1.735244   0.763364   0.044388   
...          ...        ...        ...        ...        ...        ...   
796    18.145779  17.774754  18.010370  16.633738  17.426847  17.164404   
797    18.138325  17.757563  18.009689  16.650743  17.428209  17.165398   
798    18.130869  17.740374  18.009008  16.667751  17.429573  17.166391   
799    18.123415  17.723183  18.008326  16.684757  17.430935  17.167383   
800    18.115959  17.705994  18.007645  16.701765  17.432299  17.168377   

                                                                         
time          7          8          9          10         11         12  
depth                                                                    
0       0.031504   0.062099   0.031180   0.060145   0.040893   0.158668  
1       0.034701   0.062664   0.031210   0.055454   0.040010   0.170955  
2       0.037898   0.063230   0.031239   0.050763   0.039127   0.183241  
3       0.041095   0.063796   0.031268   0.046071   0.038244   0.195527  
4       0.044292   0.064362   0.031298   0.041380   0.037361   0.207814  
...          ...        ...        ...        ...        ...        ...  
796    17.232744  17.066011  16.719736  17.397499  17.781826  18.227064  
797    17.234747  17.074242  16.747641  17.396015  17.782766  18.238298  
798    17.236750  17.082474  16.775545  17.394531  17.783707  18.249535  
799    17.238754  17.090704  16.803450  17.393047  17.784647  18.260769  
800    17.240757  17.098936  16.831354  17.391563  17.785587  18.272003  

[801 rows x 12 columns]

### now below I have some functionality, to get the MLD/WOA combined analysis..
not sure if that is relevant for you, I'll leave the functions here just in case

In [13]:
# this is a specific MLD climatology file that I have used,
# I can share it with you, or you insert one of the other MLD climatologies from IFremer

mldX = xr.open_dataset('../MLDClimatology_DeBoyerMontagut/2019_11_07_data_L3_mldmindtr02_c1m_sameasSent2MHRio20141103__Sent2BenjaminPostUcsdBremen/mld_mindtr02_l3.nc')

In [14]:
def MLD_at_latlon(lats=40,lons=-20, mlddatas=mldX, justMLD=True):
    """This function returns MLD depth across the year at a specific location"""
    #ToDO include check whether the spot is on land or water
    #Make sure the lat lon usage across everything is coherent!
    mld1 = mlddatas.drop_dims(['nlines1','nlines2','nlines3','nprf'])
    mld2 = mld1.sel(lat=lats,lon=lons,method='nearest').to_dataframe()
    #ls = landseadat.sel(lat=lats,lon=180+lons,method='nearest').variables['LSMASK'].values
    mld2['time'] = mld2.index
    mld2.index = range(1,13)
    mld2.index.names = ['time']
    #if ls!=0:
    #    return pd.DataFrame()
    if justMLD==True:
        return mld2[['mld_mindtr02_rmoutliers_smth_okrg']]#.reset_index()
    else:
        return mld2#.reset_index()

In [15]:
def ValueBelowMLD(WOAint, MLD):
    """this function filters all temperature measurements above a certain depth and takes the mean"""
    WOAint_d1 = WOAint
    WOAint_d2 = WOAint_d1
    WOAint_d2.columns = WOAint_d1.columns.droplevel()
    WOAx = WOAint_d2.reset_index()
    out = []
    print(WOAx[WOAx['depth'] > MLD[MLD['time']==1]['mld_mindtr02_rmoutliers_smth_okrg'].values[0]][1].iloc[0:10])
    for i in range(1,13):
        out.append({'Month':i,'Conc':WOAx[WOAx['depth'] > MLD[MLD['time']==i]['mld_mindtr02_rmoutliers_smth_okrg'].values[0]][i].iloc[0:10].mean()})
    return pd.DataFrame(out)

In [16]:
def MeanAboveMLD(WOAint, MLD):
    """this function filters all temperature measurements above a certain depth and takes the mean"""
    WOAint_d1 = WOAint
    WOAint_d1.columns = WOAint.columns.droplevel()
    WOAx = WOAint_d1.reset_index()
    out = []
    print(WOAx[WOAx['depth'] < MLD[MLD['time']==1]['mld_mindtr02_rmoutliers_smth_okrg'].values[0]][1])
    for i in range(1,13):
        out.append({'Month':i,'Conc':WOAx[WOAx['depth'] < MLD[MLD['time']==i]['mld_mindtr02_rmoutliers_smth_okrg'].values[0]][i].mean()})
    return pd.DataFrame(out)

In [17]:
def ReturnAnalyzedWOAdata(WOAdat, lats=40, lons=-20, mlddat=mldX, WOAvar='Temp', var='t_an'):
    
    MLD = MLD_at_latlon(lats,lons,mlddat).reset_index()
    if MLD.empty == True:
        return MLD
    else:
        WOA = WOADatInLocation(WOAdat,lats=lats,lons=lons,WOAvar=WOAvar)
        WOA_int = interpolateWOAprofiles(WOA,WOAvar)
        if WOAvar=='Temp':
            return MeanAboveMLD(WOA_int,MLD)
        elif WOAvar=='N' or WOAvar=='P' or WOAvar=='Si':
            return ValueBelowMLD(WOA_int,MLD)
        elif WOAvar=='N_above' or WOAvar=='P_above' or WOAvar=='Si_above':
            return MeanAboveMLD(WOA_int,MLD)